In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.optimize import linprog

In [ ]:
# 각 엑셀 파일을 데이터 프레임으로 불러오기
schedule_data = pd.read_excel('/content/drive/MyDrive/스마트해운물류 스터디/data/스해물_스케줄 data.xlsx')
delayed_schedule_data = pd.read_excel('/content/drive/MyDrive/스마트해운물류 스터디/data/스해물_딜레이 스케줄 data.xlsx')
vessel_data = pd.read_excel('/content/drive/MyDrive/스마트해운물류 스터디/data/스해물_선박 data.xlsx')
port_data = pd.read_excel('/content/drive/MyDrive/스마트해운물류 스터디/data/스해물_항구 위치 data.xlsx')

In [ ]:
print("Schedule Data")
schedule_data.head()

Schedule Data


,루트번호,출발항,도착항,선박명,주문량(KG),ETD,ETA,스케줄 번호
0,1,BUSAN,SAVANNAH,EVER FULL 1224E',9353.45,2025-08-07,2025-08-30,1
1,2,BUSAN,SAVANNAH,CMA CGM POINTE DU PITON 0XR8PE1MA',25671,2025-08-06,2025-09-04,2
2,3,BUSAN,SAVANNAH,COSCO SHIPPING SAKURA 031E',8463.25,2025-08-08,2025-09-08,3
3,4,BUSAN,SAVANNAH,CMA CGM POINTE-NOIRE 0XR8RE1MA',34978,2025-08-09,2025-09-09,4
4,5,BUSAN,SAVANNAH,THESEUS 1225E',19044,2025-08-12,2025-09-05,5


In [ ]:
print("\nDelayed Schedule Data")
delayed_schedule_data.head()


Delayed Schedule Data


,루트번호,출발항,도착항,선박명,딜레이 ETA,스케줄 번호
0,3,BUSAN,SAVANNAH,COSCO SHIPPING SAKURA 031E',2025-09-10,1
1,4,BUSAN,SAVANNAH,CMA CGM POINTE-NOIRE 0XR8RE1MA',2025-09-13,2
2,12,BUSAN,SAVANNAH,CMA CGM FORT DIAMANT 0XR8XE1MA',2025-09-28,3
3,18,BUSAN,SAVANNAH,COSCO FAITH 073E',2025-10-25,4
4,23,BUSAN,SAVANNAH,TOLEDO TRIUMPH 1234E',2025-11-10,5


In [ ]:
print("\nVessel Data")
vessel_data.head()


Vessel Data


,선박명,용량(TEU)
0,EVER FULL 1224E',11888
1,CMA CGM POINTE DU PITON 0XR8PE1MA',7746
2,COSCO SHIPPING SAKURA 031E',13800
3,CMA CGM POINTE-NOIRE 0XR8RE1MA',8048
4,THESEUS 1225E',14424


In [ ]:
print("\nPort Data")
port_data.head()


Port Data


,항구명,위치_위도,위치_경도
0,BUSAN,35.10160,129.0403
1,LONG BEACH,33.77538,118.1892
2,NEW YORK,40.71280,74.0060
3,SAVANNAH,32.07800,81.0912
4,HOUSTON,29.76040,95.3698


In [ ]:
# 고정값
# 컨테이너 용량(kg)
KG_PER_TEU = 30000

# 운송비(USD/TEU)
CSHIP = 1000

# 유류할증료(USD/TEU)
CBAF = 100

# ETA 패널티(USD/일)
CETA = 150

# Sets

In [ ]:
# '항구명' 컬럼의 값들을 ports 변수에 저장
P = port_data['항구명'].unique().tolist()
print("Ports:")
print(P)

# 루트 번호
R = list(range(1, 102))
print("Routes:")
print(R)

# '선박명' 컬럼의 값들을 vessels 변수에 저장
V = vessel_data['선박명'].unique().tolist()
print("Vessels:")
print(V)

# '스케줄 번호' 컬럼의 값들을 sched 변수에 저장
I = schedule_data['스케줄 번호'].tolist()
print("Schedule Numbers:")
print(I)

# '스케줄 번호' 컬럼의 값들을 delayed_sched 변수에 저장
delayed_I = delayed_schedule_data['스케줄 번호'].tolist()
print("Delayed Schedule Numbers:")
print(I)

Ports:
['BUSAN', 'LONG BEACH', 'NEW YORK', 'SAVANNAH', 'HOUSTON', 'MOBILE', 'SEATTLE']
Routes:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]
Vessels:
["EVER FULL 1224E'", "CMA CGM POINTE DU PITON 0XR8PE1MA'", "COSCO SHIPPING SAKURA 031E'", "CMA CGM POINTE-NOIRE 0XR8RE1MA'", "THESEUS 1225E'", "COSCO SHIPPING ORCHID 030E'", "EVER FORTUNE 1226E'", "CMA CGM PASSION 0XR8TE1MA'", "COSCO SHIPPING ROSE 041E'", "EVER FOCUS 1227E'", "COSCO SHIPPING AZALEA 032E'", "CMA CGM FORT DIAMANT 0XR8XE1MA'", "TAMPA TRIUMPH 1228E'", "TRITON 1229E'", "COSCO SHIPPING JASMINE 034E'", "EVER FAIR 1230E'", "EVER FAR 1231E'", "COSCO FAITH 073E'", "EVER FAVOR 1232E'", "

# Parameters

In [ ]:
O_i = schedule_data.set_index('스케줄 번호')['출발항'].to_dict()
print("Origin ports for each schedule (O_i):")
print(O_i)

Origin ports for each schedule (O_i):
{1: 'BUSAN', 2: 'BUSAN', 3: 'BUSAN', 4: 'BUSAN', 5: 'BUSAN', 6: 'BUSAN', 7: 'BUSAN', 8: 'BUSAN', 9: 'BUSAN', 10: 'BUSAN', 11: 'BUSAN', 12: 'BUSAN', 13: 'BUSAN', 14: 'BUSAN', 15: 'BUSAN', 16: 'BUSAN', 17: 'BUSAN', 18: 'BUSAN', 19: 'BUSAN', 20: 'BUSAN', 21: 'BUSAN', 22: 'BUSAN', 23: 'BUSAN', 24: 'BUSAN', 25: 'BUSAN', 26: 'BUSAN', 27: 'BUSAN', 28: 'BUSAN', 29: 'BUSAN', 30: 'BUSAN', 31: 'BUSAN', 32: 'BUSAN', 33: 'BUSAN', 34: 'BUSAN', 35: 'BUSAN', 36: 'BUSAN', 37: 'BUSAN', 38: 'BUSAN', 39: 'BUSAN', 40: 'BUSAN', 41: 'BUSAN', 42: 'BUSAN', 43: 'BUSAN', 44: 'BUSAN', 45: 'BUSAN', 46: 'BUSAN', 47: 'BUSAN', 48: 'BUSAN', 49: 'BUSAN', 50: 'BUSAN', 51: 'BUSAN', 52: 'BUSAN', 53: 'BUSAN', 54: 'BUSAN', 55: 'BUSAN', 56: 'BUSAN', 57: 'BUSAN', 58: 'BUSAN', 59: 'BUSAN', 60: 'BUSAN', 61: 'BUSAN', 62: 'BUSAN', 63: 'BUSAN', 64: 'BUSAN', 65: 'BUSAN', 66: 'BUSAN', 67: 'BUSAN', 68: 'BUSAN', 69: 'BUSAN', 70: 'BUSAN', 71: 'BUSAN', 72: 'BUSAN', 73: 'BUSAN', 74: 'BUSAN', 75: 'BUS

In [ ]:
D_i = schedule_data.set_index('스케줄 번호')['도착항'].to_dict()
print("Destination ports for each schedule (D_i):")
print(D_i)

Destination ports for each schedule (D_i):
{1: 'SAVANNAH', 2: 'SAVANNAH', 3: 'SAVANNAH', 4: 'SAVANNAH', 5: 'SAVANNAH', 6: 'SAVANNAH', 7: 'SAVANNAH', 8: 'SAVANNAH', 9: 'SAVANNAH', 10: 'SAVANNAH', 11: 'SAVANNAH', 12: 'SAVANNAH', 13: 'SAVANNAH', 14: 'SAVANNAH', 15: 'SAVANNAH', 16: 'SAVANNAH', 17: 'SAVANNAH', 18: 'SAVANNAH', 19: 'SAVANNAH', 20: 'SAVANNAH', 21: 'SAVANNAH', 22: 'SAVANNAH', 23: 'SAVANNAH', 24: 'SAVANNAH', 25: 'SAVANNAH', 26: 'SAVANNAH', 27: 'SAVANNAH', 28: 'SAVANNAH', 29: 'SAVANNAH', 30: 'SAVANNAH', 31: 'NEW YORK', 32: 'NEW YORK', 33: 'NEW YORK', 34: 'NEW YORK', 35: 'NEW YORK', 36: 'NEW YORK', 37: 'NEW YORK', 38: 'NEW YORK', 39: 'NEW YORK', 40: 'NEW YORK', 41: 'NEW YORK', 42: 'NEW YORK', 43: 'NEW YORK', 44: 'NEW YORK', 45: 'NEW YORK', 46: 'NEW YORK', 47: 'NEW YORK', 48: 'NEW YORK', 49: 'NEW YORK', 50: 'NEW YORK', 51: 'NEW YORK', 52: 'NEW YORK', 53: 'NEW YORK', 54: 'NEW YORK', 55: 'NEW YORK', 56: 'NEW YORK', 57: 'NEW YORK', 58: 'NEW YORK', 59: 'NEW YORK', 60: 'NEW YORK', 61: '

In [ ]:
V_r = schedule_data.set_index('루트번호')['선박명'].to_dict()
print("Vessels assigned to each route (V_r):")
print(V_r)

Vessels assigned to each route (V_r):
{1: "EVER FULL 1224E'", 2: "CMA CGM POINTE DU PITON 0XR8PE1MA'", 3: "COSCO SHIPPING SAKURA 031E'", 4: "CMA CGM POINTE-NOIRE 0XR8RE1MA'", 5: "THESEUS 1225E'", 6: "COSCO SHIPPING ORCHID 030E'", 7: "EVER FORTUNE 1226E'", 8: "CMA CGM PASSION 0XR8TE1MA'", 9: "COSCO SHIPPING ROSE 041E'", 10: "EVER FOCUS 1227E'", 11: "COSCO SHIPPING AZALEA 032E'", 12: "CMA CGM FORT DIAMANT 0XR8XE1MA'", 13: "TAMPA TRIUMPH 1228E'", 14: "TRITON 1229E'", 15: "COSCO SHIPPING JASMINE 034E'", 16: "EVER FAIR 1230E'", 17: "EVER FAR 1231E'", 18: "COSCO FAITH 073E'", 19: "EVER FAVOR 1232E'", 20: "COSCO SHIPPING CAMELLIA 029E'", 21: "EVER FOND 1233E'", 22: "COSCO PRIDE 085E'", 23: "TOLEDO TRIUMPH 1234E'", 24: "CMA CGM PALMYRE 0XR9BE1MA'", 25: "EVER FULL 1235E'", 26: "COSCO SHIPPING SAKURA 032E'", 27: "CMA CGM VOLGA 0XR9DE1MA'", 28: "THESEUS 1236E'", 29: "COSCO SHIPPING ORCHID 031E'", 30: "CMA CGM POINTE DU PITON 0XR9FE1MA'", 31: "OOCL BRUSSELS 063W'", 32: "EVER FULL 1224E'", 33: "COS

In [ ]:
Q_r = schedule_data.groupby('루트번호')['주문량(KG)'].sum().to_dict()
print("Total order quantity for each route (Q_r):")
print(Q_r)

Total order quantity for each route (Q_r):
{1: 9353.45, 2: 25671, 3: 8463.25, 4: 34978, 5: 19044, 6: 1646.4, 7: 27478.84, 8: 8156, 9: 9621.06, 10: 25860.38, 11: 16523, 12: 8320, 13: 8062.5, 14: 17977.5, 15: 21446.54, 16: 56631, 17: 17369, 18: 72582, 19: 9015.38, 20: 8424.89, 21: 38519.2, 22: 84623.97, 23: 41904.22, 24: 37877.5, 25: 64818.9, 26: 53304.7, 27: 98424.89, 28: 23215.19, 29: 9608.1, 30: 76005.36, 31: 43920.61, 32: 14016.9, 33: 81605.54, 34: 29504.3, 35: 61901.9, 36: 14818.12, 37: 9403.89, 38: 8401, 39: 37810, 40: 93923.9, 41: 28824, 42: 66005.36, 43: 53215.19, 44: 9624.3, 45: 73215.15, 46: 81905.76, 47: 42289.23, 48: 14202.92, 49: 92008.99, 50: 78300.62, 51: 39044, 52: 81646.4, 53: 23987.65, 54: 8471.23, 55: 61235.89, 56: 56156, 57: 76523.46, 58: 11879.3, 59: 44759, 60: 89235.67, 61: 72108.34, 62: 31854.4, 63: 52885.21, 64: 99000, 65: 8231.56, 70: 29564.68, 71: 13498.24, 72: 66676.64, 73: 84940, 74: 43179, 75: 69987, 76: 71424.98, 77: 53666.25, 78: 23186.47, 79: 31712.99, 80:

In [ ]:
ETD_i = schedule_data.set_index('스케줄 번호')['ETD'].to_dict()
print("Estimated Departure Times (ETD_i):")
print(ETD_i)

ETA_i = schedule_data.set_index('스케줄 번호')['ETA'].to_dict()
print("\nEstimated Arrival Times (ETA_i):")
print(ETA_i)

Estimated Departure Times (ETD_i):
{1: Timestamp('2025-08-07 00:00:00'), 2: Timestamp('2025-08-06 00:00:00'), 3: Timestamp('2025-08-08 00:00:00'), 4: Timestamp('2025-08-09 00:00:00'), 5: Timestamp('2025-08-12 00:00:00'), 6: Timestamp('2025-08-16 00:00:00'), 7: Timestamp('2025-08-21 00:00:00'), 8: Timestamp('2025-08-18 00:00:00'), 9: Timestamp('2025-08-20 00:00:00'), 10: Timestamp('2025-08-23 00:00:00'), 11: Timestamp('2025-08-24 00:00:00'), 12: Timestamp('2025-08-28 00:00:00'), 13: Timestamp('2025-08-30 00:00:00'), 14: Timestamp('2025-09-06 00:00:00'), 15: Timestamp('2025-09-11 00:00:00'), 16: Timestamp('2025-09-14 00:00:00'), 17: Timestamp('2025-09-21 00:00:00'), 18: Timestamp('2025-09-22 00:00:00'), 19: Timestamp('2025-09-28 00:00:00'), 20: Timestamp('2025-09-28 00:00:00'), 21: Timestamp('2025-10-05 00:00:00'), 22: Timestamp('2025-10-05 00:00:00'), 23: Timestamp('2025-10-12 00:00:00'), 24: Timestamp('2025-10-16 00:00:00'), 25: Timestamp('2025-10-19 00:00:00'), 26: Timestamp('2025-10-

In [ ]:
RETA_i = delayed_schedule_data.set_index('스케줄 번호')['딜레이 ETA'].to_dict()
print("Actual Arrival Times (RETA_i):")
print(RETA_i)

KeyError: '딜레이 ETA'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Iterate through ETA_i and add to RETA_i if not present
for schedule_num in ETA_i:
    if schedule_num not in RETA_i:
        RETA_i[schedule_num] = ETA_i[schedule_num]

RETA_i = dict(sorted(RETA_i.items()))

print("Actual Arrival Times (RETA_i_sorted):")
print(RETA_i)

In [ ]:
import math

D_ab = {}
for r in R:
  if r in Q_r: # Check if the route exists in Q_r
    D_ab[r] = math.ceil(Q_r[r] / KG_PER_TEU)

print("Order quantity in TEU for each route (D_ab):")
print(D_ab)

In [ ]:
CAP_v = vessel_data.set_index('선박명')['용량(TEU)'].to_dict()
print("Vessel capacities (CAP_v):")
print(CAP_v)

In [ ]:
CAP_v_r = {}
for r in V_r:
    vessel_name = V_r[r]
    if vessel_name in CAP_v:
        CAP_v_r[r] = CAP_v[vessel_name]
    else:
        # Handle cases where a vessel in V_r is not found in CAP_v if necessary
        CAP_v_r[r] = None # Or some other default value

print("Capacity of the vessel assigned to each route (CAP_v_r):")
print(CAP_v_r)

In [ ]:
DELAY_i = {}
for i in I:
    if i in ETA_i and i in RETA_i:
        delay_days = (RETA_i[i] - ETA_i[i]).days
        DELAY_i[i] = max(0, delay_days)
    else:
        # Handle cases where a schedule number is not in both dictionaries if necessary
        DELAY_i[i] = 0 # Assuming no delay if data is missing

print("Delay in days for each schedule (DELAY_i):")
print(DELAY_i)

# Decision Variables

In [ ]:
# Step 3: Define Decision Variables (Time-Phased Inventory Model)

# Based on user's clarification and assuming the PDF defines:
# xF_i: Number of full containers transported on schedule i (for i in I)
# xE_i: Number of empty containers transported on schedule i (for i in I)
# y_ip: Number of empty containers at port p after processing schedule i (for i in I, p in P)

# Define the number of schedules and ports
num_schedules = len(I) # Assuming I is defined (list of schedule numbers)
num_ports = len(P)   # Assuming P is defined (list of port names)

# Define the total number of variables:
# xF_i (num_schedules)
# xE_i (num_schedules)
# y_ip (num_schedules * num_ports)
num_variables = num_schedules + num_schedules + (num_schedules * num_ports)

# Create a list of decision variable names for clarity (optional)
# The order in this list should match the order in the flattened variable array for linprog
variable_names = [f'xF_{i}' for i in I] + \
                   [f'xE_{i}' for i in I] + \
                   [f'y_{i}_{p}' for i in I for p in P]


print("Number of schedules (num_schedules):", num_schedules)
print("Number of ports (num_ports):", num_ports)
print("Total number of decision variables (num_variables):", num_variables)
# print("Decision variable names (first few and last few):")
# print(variable_names[:5], "...", variable_names[-5:])

# LP Model

In [ ]:
# Step 4: Formulate the Objective Function (Time-Phased Inventory Model)

# Analyze the PDF to get the coefficients for xF_i, xE_i, and y_ip
# Assuming the objective is to minimize total cost.
# Based on typical container shipping LPs and previous discussions:
# Cost for xF_i = CSHIP + CBAF + CETA * DELAY_i
# Cost for xE_i = Assuming a cost for transporting empty containers (let's use CEMPTY_SHIP)
# Cost for y_ip = Assuming an inventory holding cost at port p after schedule i (let's use CHOLD_p_i)
# Note: The holding cost might depend on the port p and/or the time step (schedule i).
# For simplicity, let's assume a constant holding cost per TEU per "schedule period" (CHOLD).

# Define placeholder costs if they are not explicitly defined in the PDF or previous cells
# (Replace these with actual values from the PDF if available)
CEMPTY_SHIP = CSHIP + CBAF # Assuming same transport cost for empty as full containers
CHOLD = 10 # Example holding cost per empty container per schedule period

# The coefficients for the objective function need to be a 1D array
# The order should match the order of decision variables: [xF_1, ..., xF_n, xE_1, ..., xE_n, y_1_p1, ..., y_n_pm]
# num_schedules and num_ports are assumed to be defined

c_obj = []

# Coefficients for xF_i
for i in I: # Assuming I is defined
    # Ensure DELAY_i is defined for this schedule
    if i in DELAY_i: # Assuming DELAY_i is defined
        c_obj.append(CSHIP + CBAF + CETA * DELAY_i[i]) # Assuming CSHIP, CBAF, CETA are defined
    else:
        # Assume no delay penalty if delay data is missing
        c_obj.append(CSHIP + CBAF)

# Coefficients for xE_i
for i in I: # Assuming I is defined
    # Assuming the cost for xE_i is CEMPTY_SHIP
    c_obj.append(CEMPTY_SHIP) # Using the placeholder cost

# Coefficients for y_ip
# Assuming a holding cost CHOLD for each y_ip
for i in I:
    for p in P: # Assuming P is defined
        c_obj.append(CHOLD) # Using the placeholder holding cost

# Convert to numpy array
c_obj = np.array(c_obj)

print("Objective function coefficients (c_obj):")
# print(c_obj) # Avoid printing very large arrays
print("c_obj shape:", c_obj.shape)
print("First 10 elements:", c_obj[:10]) # Print first few elements
print("Last 10 elements:", c_obj[-10:]) # Print last few elements


# Note: The exact objective function and costs should be verified from the PDF.
# Placeholder costs (CEMPTY_SHIP, CHOLD) are used if not specified in the PDF.

In [ ]:
# Define I0_p (Initial empty containers at each port) - Using values from previous scenario
# Assuming P is defined
I0_p = {p: 0 for p in P}
I0_p['BUSAN'] = 50000
I0_p['LONG BEACH'] = 30000
I0_p['NEW YORK'] = 100000
I0_p['SAVANNAH'] = 20000
I0_p['HOUSTON'] = 10000
I0_p['MOBILE'] = 10000
# Note: 'SEATLE' had a typo in the previous definition, assuming it should be 'SEATTLE'
if 'SEATTLE' in P:
    I0_p['SEATTLE'] = 10000
else:
     # Handle case if 'SEATTLE' is not in P
     print("Warning: 'SEATTLE' not found in the list of ports (P). Skipping I0_p assignment for SEATTLE.")

print("Initial number of empty containers at each port (I0_p):")
print(I0_p)

In [ ]:
# Step 5: Formulate Constraints (Time-Phased Inventory Model)

# Initialize constraint matrices/vectors as lists
A_ub = []
b_ub = []
A_eq = []
b_eq = []

# Assuming num_schedules, num_ports, num_variables are defined
# Assuming I, P, R, D_ab, CAP_v_r, O_i, D_i, I0_p, schedule_data are defined

# Assuming decision variables order: [xF_1..n, xE_1..n, y_1_p1..m, y_2_p1..m, ..., y_n_p1..m]
xF_indices = {i: I.index(i) for i in I}
xE_indices = {i: num_schedules + I.index(i) for i in I}
y_indices = {(i, p): 2 * num_schedules + I.index(i) * num_ports + P.index(p) for i in I for p in P}

# Constraint: Initial Inventory Balance (Linking I0_p to y_1p)
# y_1p = I0_p[p] + arrivals(schedule 1 at p) - departures(schedule 1 from p)
# Rearrange: y_1p - arrivals(schedule 1 at p) + departures(schedule 1 from p) = I0_p[p]
# Note: This assumes the first schedule I[0] is the first event affecting inventory.
# A more robust model would order events by time.
first_schedule = I[0]
for p in P:
    row = [0] * num_variables
    # Coefficient for y_1p
    row[y_indices[(first_schedule, p)]] = 1

    # Arrivals from the first schedule at port p
    if first_schedule in D_i and D_i[first_schedule] == p:
        row[xF_indices[first_schedule]] -= 1 # - (xF_1)
        row[xE_indices[first_schedule]] -= 1 # - (xE_1)

    # Departures from port p by the first schedule
    if first_schedule in O_i and O_i[first_schedule] == p:
        row[xF_indices[first_schedule]] += 1 # + (xF_1)
        row[xE_indices[first_schedule]] += 1 # + (xE_1)

    A_eq.append(row)
    b_eq.append(I0_p.get(p, 0)) # Right hand side is I0_p[p]


# Constraint: Time-Phased Inventory Balance (for schedules i > 1)
# y_ip = y_(i-1)p + arrivals(schedule i at p) - departures(schedule i from p)
# Rearrange: y_ip - y_(i-1)p - arrivals(schedule i at p) + departures(schedule i from p) = 0
for i_idx in range(1, num_schedules): # Iterate from the second schedule
    current_schedule = I[i_idx]
    previous_schedule = I[i_idx - 1]
    for p in P:
        row = [0] * num_variables
        # Coefficient for y_ip
        row[y_indices[(current_schedule, p)]] = 1

        # Coefficient for y_(i-1)p
        row[y_indices[(previous_schedule, p)]] = -1

        # Arrivals from the current schedule at port p
        if current_schedule in D_i and D_i[current_schedule] == p:
            row[xF_indices[current_schedule]] -= 1 # - (xF_i)
            row[xE_indices[current_schedule]] -= 1 # - (xE_i)

        # Departures from port p by the current schedule
        if current_schedule in O_i and O_i[current_schedule] == p:
            row[xF_indices[current_schedule]] += 1 # + (xF_i)
            row[xE_indices[current_schedule]] += 1 # + (xE_i)

        A_eq.append(row)
        b_eq.append(0) # Right hand side is 0


# Constraint: Demand Fulfillment for each route r (Assuming from PDF)
# Sum of xF_i for schedules i in route r must be >= D_ab[r]
# Convert to <= form: -Sum(xF_i for i in route r) <= -D_ab[r]
for r in R:
    if r in D_ab:
        row = [0] * num_variables
        schedules_in_route = schedule_data[schedule_data['루트번호'] == r]['스케줄 번호'].tolist()
        for i in I:
            if i in schedules_in_route:
                 row[xF_indices[i]] = -1
        A_ub.append(row)
        b_ub.append(-D_ab[r])


# Constraint: Vessel Capacity for each route r (Assuming from PDF)
# Sum of xF_i + Sum of xE_i for schedules i in route r must be <= CAP_v_r[r]
for r in R:
    if r in CAP_v_r and CAP_v_r[r] is not None:
        row = [0] * num_variables
        schedules_in_route = schedule_data[schedule_data['루트번호'] == r]['스케줄 번호'].tolist()
        for i in I:
            if i in schedules_in_route:
                row[xF_indices[i]] = 1
                row[xE_indices[i]] = 1
        A_ub.append(row)
        b_ub.append(CAP_v_r[r])

# New Constraint: Total Empty Container Transport on a Route is a Fraction of Vessel Capacity
# Sum of xE_i for schedules i in route r = theta * CAP_v_r[r] for each route r
theta = 0.001
for r in R:
    if r in CAP_v_r and CAP_v_r[r] is not None:
        row = [0] * num_variables
        schedules_in_route = schedule_data[schedule_data['루트번호'] == r]['스케줄 번호'].tolist()
        for i in I:
            if i in schedules_in_route:
                 # The index for xE_i is num_schedules + I.index(i)
                xE_index = num_schedules + I.index(i)
                row[xE_index] = 1 # Coefficient is 1 for equality constraint

        # Right-hand side is theta * CAP_v_r for route r
        rhs = theta * CAP_v_r[r]
        A_eq.append(row)
        b_eq.append(rhs)
    else:
        # Handle cases where capacity data is missing for a route if necessary
        print(f"Warning: Capacity data missing for route {r}. Skipping total empty container constraint.")


# Convert to numpy arrays
A_ub = np.array(A_ub)
b_ub = np.array(b_ub)
A_eq = np.array(A_eq)
b_eq = np.array(b_eq)


print("A_ub shape:", A_ub.shape)
print("b_ub shape:", b_ub.shape)
print("A_eq shape:", A_eq.shape)
print("b_eq shape:", b_eq.shape)

# Note: Additional constraints from the PDF should be added here following the same pattern.
# The exact constraints from the PDF need to be verified.

In [ ]:
# Step 6: Set Variable Bounds

# Bounds for decision variables (xF_i >= 0, xE_i >= 0, y_ip >= 0)
# Assuming num_variables is defined from Step 3
bounds = [(0, None)] * num_variables

print("Bounds for decision variables:")
print(bounds[:5], "...", bounds[-5:]) # Print only first/last few bounds
print("Total number of bounds:", len(bounds))

In [ ]:
# Step 7: Solve the LP Problem

from scipy.optimize import linprog

# Solve the linear programming problem
result = linprog(c_obj, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)

# Print the result of the solver
print(result)

In [ ]:
# Step 8: Analyze and Output Results

# Check if the optimization was successful
if result.success:
    print("Optimization successful!")
    print(f"Optimal total cost: {result.fun}") # Print the optimal objective function value

    # Extract the optimal values of the decision variables
    optimal_x = result.x

    # Define the number of schedules and ports again for clarity
    num_schedules = len(I) # Assuming I is defined
    num_ports = len(P) # Assuming P is defined

    # Extract xF_i values and convert to integers
    xF_optimal = {}
    for idx, i in enumerate(I):
        xF_optimal[i] = int(round(optimal_x[idx])) # Round before converting

    # Extract xE_i values and convert to integers
    xE_optimal = {}
    for idx, i in enumerate(I):
        xE_optimal[i] = int(round(optimal_x[num_schedules + idx])) # Round before converting

    # Extract y_ip values and convert to integers, organize into a dictionary for DataFrame
    y_ip_optimal = {}
    y_ip_start_index = 2 * num_schedules
    for i_idx, i in enumerate(I):
        y_ip_optimal[i] = {}
        for p_idx, p in enumerate(P):
            variable_index = y_ip_start_index + i_idx * num_ports + p_idx
            y_ip_optimal[i][p] = int(round(optimal_x[variable_index])) # Round before converting

    print("\nOptimal values for xF_i (Full containers transported per schedule):")
    # Print dictionary directly
    print(xF_optimal)

    print("\nOptimal values for xE_i (Empty containers transported per schedule):")
    # Print dictionary directly
    print(xE_optimal)

    print("\nOptimal values for y_ip (Empty containers at port p after processing schedule i):")
    # Convert dictionary of dictionaries to DataFrame for better display
    y_ip_df = pd.DataFrame.from_dict(y_ip_optimal, orient='index')
    y_ip_df.index.name = 'Schedule Number'
    y_ip_df.columns.name = 'Port'
    display(y_ip_df)


else:
    print("Optimization failed.")
    print("Status:", result.status)
    print("Message:", result.message)